In [1]:
import glob
import rasterio as rio
import numpy as np
from osgeo import gdal
import os
from collections import defaultdict

In [2]:
from General_Raster_Funcs.RasterTiling import getTiles
from General_Raster_Funcs.TiffManagement import *

In [3]:
from MODIS_Raster_Funcs.SynopticData import MonthlyStatCalculator

## Set up file locations - edit then run each cell

In [5]:
inFilePattern = r"F:\MOD11A2_Input_Mosaics\Day\A2015*.tif"

In [9]:
tileDir = r"C:\temp\test\tiles"
outDir = r"C:\temp\test\merged"
what = "TestDA"

In [8]:
# Specify the height of each tile - depends on available memory.
# The algorithm needs around idealSlice * fullWidth * 80 bytes of RAM
# so with global 1k images (43200px wide), a slice of 7168 needs 
# around 25Gb RAM.
# The rasters have tilesize 256 (or a multiple thereof) so pick a size
# that is a multiple of this where possible for most efficient access
idealSlice = 7168

# alter to suit the images
fullWidth = 43200
fullHeight = 21600

# alter to whatever you want
outNdv = -9999



### Run the following cells unaltered to configure the remaining inputs

In [4]:
# build a dictionary mapping day of year to month of year, only required for the 
# day numbers that the 8-daily MODIS data occurs on
# generate this in excel with =CONCATENATE(DAYNUM,":",MONTH(DAYNUM),", ")
daymonths = {1:1, 9:1, 17:1, 25:1, 33:2, 41:2, 49:2, 57:2, 65:3, 73:3, 81:3, 89:3, 97:4, 
             105:4, 113:4, 121:4, 129:5, 137:5, 145:5, 153:6, 161:6, 169:6, 177:6, 185:7, 
             193:7, 201:7, 209:7, 217:8, 225:8, 233:8, 241:8, 249:9, 257:9, 265:9, 273:9, 
             281:10, 289:10, 297:10, 305:10, 313:11, 321:11, 329:11, 337:12, 345:12, 353:12, 
             361:12}
# swap to build list of days for each month
monthDays = defaultdict(list)
for d,m in daymonths.iteritems():
    monthDays[m].append(d)
    

In [6]:
# build a list of MODIS files available for each day-of-year, based on the 
# year / julian day that's encoded in the filenames such as "A2015009_LST_Day.tif"
years = defaultdict(int)
days = defaultdict(int)
dayfiles = defaultdict(list)
for fn in glob.glob(inFilePattern):
    datestr = os.path.basename(fn).split('_')[0][1:]
    yr = int(datestr[:4])
    years[yr] +=1
    day = int(datestr[4:])
    days[day] +=1
    month = daymonths[day]
    dayfiles[day].append(fn)

    

In [7]:
globalGT = None
globalProj = None
stats = ['Count', 'Mean', 'SD']
# work out the tiles we'll work in. We'll work with full-width slices for 
# now. 
slices = sorted(list(set([s[1] for s in getTiles(fullWidth, fullHeight, idealSlice)])))

In [10]:
fnGetter = lambda what, when, stat, where:(
    str(what) + "_" + str(when) + "_" + str(stat)
    + "_" + str(where)+ ".tif")

## Run this cell to calculate the results and save to tiled tiffs

In [11]:

for t,b in slices:
    # initialise reader
    sliceHeight = b-t
    
    statsCalculator = MonthlyStatCalculator(sliceHeight, width, outNdv)
    sliceGT = None
    sliceProj = None
    
    print str((t,b))
    for month, days in monthDays.iteritems():
        # for each calendar day of this synoptic month 
        print "\tMonth "+str(month)
        for day in days:
            # for each file on this calendar day (i.e. one per year)
            print"\t\tDay "+str(day)
            for dayfile in dayfiles[day]:
                # add slice
                data, myGT, myProj, thisNdv = ReadAOI_PixelLims(dayfile, None, (t,b))
                if sliceGT is None:
                    sliceGT = myGT
                    sliceProj = myProj
                else:
                    assert sliceGT == myGT
                    assert sliceProj == myProj
                # add the data to the running calculator
                statsCalculator.addFile(data, month, thisNdv)
        # get and save the results for this synoptic month
        monthResults = statsCalculator.emitMonth()
        SaveLZWTiff(monthResults['count'], outNdv, sliceGT, sliceProj, tileDir,
                   fnGetter(what, "M" + str(month).zfill(2), "Count", t))
        SaveLZWTiff(monthResults['mean'], outNdv, sliceGT, sliceProj, tileDir,
                   fnGetter(what, "M" + str(month).zfill(2), "Mean", t))
        SaveLZWTiff(monthResults['sd'], outNdv, sliceGT, sliceProj, tileDir,
                   fnGetter(what, "M" + str(month).zfill(2), "SD", t))
    statsCalculator = None
    
    # get and save the overall synoptic result
    overallResults = statsCalculator.emitTotal()
    SaveLZWTiff(overallResults['count'], outNdv, sliceGT, sliceProj, tileDir,
        fnGetter(what, "Overall", "Count", t))
    SaveLZWTiff(overallResults['mean'], outNdv, sliceGT, sliceProj, tileDir,
        fnGetter(what, "Overall", "Mean", t))
    SaveLZWTiff(overallResults['sd'], outNdv, sliceGT, sliceProj, tileDir,
        fnGetter(what, "Overall", "SD", t))
        
        

(0, 5400)
	Month 1
		Day 1
		Day 9
		Day 25
		Day 17
	Month 2
		Day 33
		Day 49
		Day 57
		Day 41
	Month 3
		Day 65
		Day 81
		Day 89
		Day 73
	Month 4
		Day 97
		Day 105
		Day 113
		Day 121
	Month 5
		Day 129
		Day 137
		Day 145
	Month 6
		Day 153
		Day 161
		Day 169
		Day 177
	Month 7
		Day 185
		Day 193
		Day 201
		Day 209
	Month 8
		Day 217
		Day 225
		Day 233
		Day 241
	Month 9
		Day 257
		Day 273
		Day 265
		Day 249
	Month 10
		Day 297
		Day 305
		Day 281
		Day 289
	Month 11
		Day 329
		Day 313
		Day 321
	Month 12
		Day 345
		Day 353
		Day 361
		Day 337
(5400, 10800)
	Month 1
		Day 1
		Day 9
		Day 25
		Day 17
	Month 2
		Day 33
		Day 49
		Day 57
		Day 41
	Month 3
		Day 65
		Day 81
		Day 89
		Day 73
	Month 4
		Day 97
		Day 105
		Day 113
		Day 121
	Month 5
		Day 129
		Day 137
		Day 145
	Month 6
		Day 153
		Day 161
		Day 169
		Day 177
	Month 7
		Day 185
		Day 193
		Day 201
		Day 209
	Month 8
		Day 217
		Day 225
		Day 233
		Day 241
	Month 9
		Day 257
		Day 273
		Day 265
		Day 249
	Mon

## Run this cell to merge the tiles to global outputs

In [17]:
import subprocess
vrtBuilder = "gdalbuildvrt {0} {1}"
transBuilder = "gdal_translate -of GTiff -co COMPRESS=LZW "+\
    "-co PREDICTOR=2 -co TILED=YES -co SPARSE_OK=TRUE -co BIGTIFF=YES "+\
    "--config GDAL_CACHEMAX 8000 {0} {1}"

vrts = []
for stat in stats:
    for month in sorted(monthDays.keys()):
        tiffWildCard = fnGetter(what, 'M'+str(month).zfill(2), stat, "*")
        sliceTiffs = os.path.join(tileDir, tiffWildCard)
        vrtName = "Month_" + str(month).zfill(2) + "_" + stat + ".vrt"
        vrtFile = os.path.join(outDir, vrtName)
        vrtCommand = vrtBuilder.format(vrtFile, 
                                      sliceTiffs)
        print vrtCommand
        vrts.append(vrtFile)
        subprocess.call(vrtCommand)
    tiffWildCard = fnGetter(what, "Overall", stat, "*")
    sliceTiffs = os.path.join(tileDir, tiffWildCard)
    vrtName = "Overall_" + stat + ".vrt"
    vrtFile = os.path.join(outDir, vrtName)
    vrtCommand = vrtBuilder.format(vrtFile, 
                                      sliceTiffs)
    print vrtCommand
    vrts.append(vrtFile)
    subprocess.call(vrtCommand)
for vrt in vrts:
    transCommand = transBuilder.format(vrt, vrt.replace('vrt', 'tif'))
    print transCommand
    subprocess.call(transCommand)

gdalbuildvrt C:\temp\test\merged\Month_01_Count.vrt C:\temp\test\tiles\TestDA_M01_Count_*.tif
gdalbuildvrt C:\temp\test\merged\Month_02_Count.vrt C:\temp\test\tiles\TestDA_M02_Count_*.tif
gdalbuildvrt C:\temp\test\merged\Month_03_Count.vrt C:\temp\test\tiles\TestDA_M03_Count_*.tif
gdalbuildvrt C:\temp\test\merged\Month_04_Count.vrt C:\temp\test\tiles\TestDA_M04_Count_*.tif
gdalbuildvrt C:\temp\test\merged\Month_05_Count.vrt C:\temp\test\tiles\TestDA_M05_Count_*.tif
gdalbuildvrt C:\temp\test\merged\Month_06_Count.vrt C:\temp\test\tiles\TestDA_M06_Count_*.tif
gdalbuildvrt C:\temp\test\merged\Month_07_Count.vrt C:\temp\test\tiles\TestDA_M07_Count_*.tif
gdalbuildvrt C:\temp\test\merged\Month_08_Count.vrt C:\temp\test\tiles\TestDA_M08_Count_*.tif
gdalbuildvrt C:\temp\test\merged\Month_09_Count.vrt C:\temp\test\tiles\TestDA_M09_Count_*.tif
gdalbuildvrt C:\temp\test\merged\Month_10_Count.vrt C:\temp\test\tiles\TestDA_M10_Count_*.tif
gdalbuildvrt C:\temp\test\merged\Month_11_Count.vrt C:\temp\

In [ ]:
ovcommand = "gdaladdo -ro --config COMPRESS_OVERVIEW LZW --config USE_RRD NO " +\
        "--config TILED YES {0} 2 4 8 16 32 64 128 256 --config GDAL_CACHEMAX 8000"
statcommand = "gdalinfo -stats {0} >nul"
subprocess.call(vrtBuilder)
import subprocess

    for f in dirTileList:
        os.remove(f)

### Code to run: Generate the statistics as 13-band rasters

###### One band for each month, the 13th band is the overall (daily) mean / SD / count

In [ ]:
# pick a slice size that aligns to 256 pixel tiff tile size for optimal access.
# 5376 gives 4 slices and seems much quicker overall than 10752 (2 slices)
calcStats(43200,21600,5376,'EVI',r'E:\MCD43B4\MCD43B4_Indices')

###### Generate the "balanced" mean as the mean of the 12 monthly bands

In [ ]:
calcBalancedMeans('EVI_WinterCut_OcclusionMasked_Nbr2')

###### Same for all the others...

In [ ]:
calcStats(43200,21600,5376,'Day',r'F:\MOD11A2\MOD11A2_Data')

In [ ]:
calcBalancedMeans('Day')

In [ ]:

calcStats(43200,21600,5376,'Night',r'F:\MOD11A2\MOD11A2_Data')

In [ ]:
calcBalancedMeans('Night')

In [ ]:
calcStats(43200,21600,5376,'TCW',r'E:\MCD43B4\MCD43B4_Indices')

In [ ]:
calcBalancedMeans('TCW')

In [ ]:
calcStats(43200,21600,10752,'TCB',r'E:\MCD43B4\MCD43B4_Indices')

In [ ]:
calcBalancedMeans('TCB')

In [ ]:
calcStats(43200,21600,5376,'LST_DiurnalDifference',r'E:\MOD11A2_DiurnalDiffs_Output\LST_Diurnal_Diffs\Output_8day')

In [ ]:
calcBalancedMeans('LST_DiurnalDifference')

### Now translate the enormous output tiffs into compressed files

Translate each in turn with the following commands, run at the command line in the C: temp directory (replace "TC\*" with appropriate pattern)

```
for /F "usebackq tokens=1 delims=." %f in (`dir /B TC*.tif`) do (
gdal_translate -of GTiff -co "COMPRESS=LZW" -co "TILED=YES" -co "PREDICTOR=2" -co "SPARSE_OK=TRUE" -co "BIGTIFF=YES" -co "INTERLEAVE=BAND" %f.tif G:\StatsOutput\%f.tif
)

for /F "usebackq tokens=1 delims=." %f in (`dir /B *Monthly_*`) do (
gdal_translate -b 13 -of GTiff -co "COMPRESS=LZW" -co "TILED=YES" -co "PREDICTOR=2" -co "SPARSE_OK=TRUE" %f.tif G:\StatsOutput\%f_Overall_From_Daily.tif
)
```

Optionally now add overviews at command line to save arcmap the bother

```
for %f in (*.tif) do (
gdaladdo -ro --config COMPRESS_OVERVIEW DEFLATE --config INTERLEAVE_OVERVIEW PIXEL --config BIGTIFF_OVERVIEW YES %f 2 4 8 16 32 64
)
```